In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import json
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import datetime
import time
from pyspark.sql.functions import split
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.appName("pyspark-notebook").getOrCreate()

In [3]:
df=spark.read.csv(r"C:\Users\Anergha\Documents\LTE_cell_availability_dataset.csv",header='True')

In [4]:
#df.columns

In [5]:
#df.show(1)

In [6]:
#df.printSchema()

In [7]:
df1=(df.withColumnRenamed('pm_data_source.nhg_alias','pm_data_source_nhg_alias')
.withColumnRenamed('pm_data_source.end_timestamp','pm_data_source_end_timestamp')
.withColumnRenamed('pm_data_source.hw_alias','pm_data_source_hw_alias')
.withColumnRenamed('pm_data_source.hw_id','pm_data_source_hw_id')
.withColumnRenamed('pm_data_source.nhg_alias','pm_data_source_nhg_alias')
.withColumnRenamed('pm_data.LTE_Cell_Avail_M8020C4','pm_data_LTE_Cell_Avail_M8020C4')
.withColumnRenamed('pm_data.LTE_Cell_Avail_M8020C3','pm_data_LTE_Cell_Avail_M8020C3')
.withColumnRenamed('pm_data.LTE_Cell_Avail_M8020C6','pm_data_LTE_Cell_Avail_M8020C6')
.withColumnRenamed('pm_data_source.dn','pm_data_source_dn'))  

In [8]:
df1=(df1.withColumn("pm_data_LTE_Cell_Avail_M8020C3",df1.pm_data_LTE_Cell_Avail_M8020C3.cast('int'))\
.withColumn("pm_data_LTE_Cell_Avail_M8020C4",df1.pm_data_LTE_Cell_Avail_M8020C4.cast('int'))\
.withColumn("pm_data_LTE_Cell_Avail_M8020C6",df1.pm_data_LTE_Cell_Avail_M8020C6.cast('int')))

In [9]:
df1.printSchema()

root
 |-- _id: string (nullable = true)
 |-- _score: string (nullable = true)
 |-- _type: string (nullable = true)
 |-- pm_data_source_dn: string (nullable = true)
 |-- pm_data_source.edge_alias: string (nullable = true)
 |-- pm_data_source.edge_hostname: string (nullable = true)
 |-- pm_data_source.edge_id: string (nullable = true)
 |-- pm_data_source_end_timestamp: string (nullable = true)
 |-- pm_data_source_hw_alias: string (nullable = true)
 |-- pm_data_source_hw_id: string (nullable = true)
 |-- pm_data_source_nhg_alias: string (nullable = true)
 |-- pm_data_source.nhg_id: string (nullable = true)
 |-- pm_data_source.serial_no: string (nullable = true)
 |-- pm_data_source.slice_id: string (nullable = true)
 |-- pm_data_source.technology: string (nullable = true)
 |-- pm_data_source.timestamp: string (nullable = true)
 |-- pm_data.LTE_Cell_Avail_M8020C0: string (nullable = true)
 |-- pm_data.LTE_Cell_Avail_M8020C1: string (nullable = true)
 |-- pm_data.LTE_Cell_Avail_M8020C10: str

In [10]:
df2=df1.groupBy('pm_data_source_end_timestamp','pm_data_source_nhg_alias','pm_data_source_dn','pm_data_source_hw_alias')\
.agg(sum("pm_data_LTE_Cell_Avail_M8020C4").alias('sum_M8020C4'),\
    sum("pm_data_LTE_Cell_Avail_M8020C6").alias('sum_M8020C6'),\
   sum("pm_data_LTE_Cell_Avail_M8020C3").alias('sum_M8020C3'))

In [1]:
df2.show(5,False)

In [38]:
df3=df2.withColumn('LTE_5750a',(100*col('sum_M8020C3')/col('sum_M8020C6')))

In [2]:
df3.show(5,False)

In [ ]:
df4 = df3.filter(col('pm_data_source_dn').contains('LNCEL'))

In [ ]:
#df2.write.csv(path=)